In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
spark = SparkSession.builder.appName('sparkdf').getOrCreate()
def create_session():
    spk = SparkSession.builder \
        .master("local") \
        .appName("Filter_values.com") \
        .getOrCreate()
    return spk
spark = SparkSession.builder.appName('Joins').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/23 21:44:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
# Abrindo o arquivo de vendas no formato parquet
vendas = spark.read.load ("/Users/mayaraevillyn/Downloads/download/Atividades/Vendas.parquet")
vendas.show()

+--------+----------+---------+---------+--------+
|VendasID|VendedorID|ClienteID|     Data|   Total|
+--------+----------+---------+---------+--------+
|       1|         1|       91| 1/1/2019|  8053.6|
|       2|         6|      185| 1/1/2020|   150.4|
|       3|         7|       31| 2/1/2020|  6087.0|
|       4|         5|       31| 2/1/2019| 13828.6|
|       5|         5|       31| 3/1/2018|26096.66|
|       6|         5|       31| 4/1/2020| 18402.0|
|       7|         5|       31| 6/1/2019|  7524.2|
|       8|         5|      186| 6/1/2019| 12036.6|
|       9|         7|       91| 6/1/2020| 2804.75|
|      10|         2|      202| 6/1/2020|  8852.0|
|      11|         7|       58| 8/1/2019|16545.25|
|      12|         7|       58| 9/1/2018|11411.88|
|      13|         7|       58|10/1/2019| 15829.7|
|      14|         3|      249|12/1/2020| 6154.36|
|      15|         4|      249|12/1/2018| 3255.08|
|      16|         7|      192|13/1/2020| 2901.25|
|      17|         2|       79|

In [18]:
# Abrindo o arquivo de clientes também no formato parquet
clientes = spark.read.load("/Users/mayaraevillyn/Downloads/download/Atividades/Clientes.parquet")
clientes.show()

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
|        6|       Adérito Bahía|    MA|     M|  Silver|
|        7|       Aida Dorneles|    RN|     F|  Silver|
|        8|   Alarico Quinterno|    AC|     M|  Silver|
|        9|    Alberto Cezimbra|    AM|     M|  Silver|
|       10|    Alberto Monsanto|    RN|     M|    Gold|
|       11|       Albino Canela|    AC|     M|  Silver|
|       12|     Alceste Varanda|    RR|     F|  Silver|
|       13|  Alcides Carvalhais|    RO|     M|  Silver|
|       14|        Aldo Martins|    GO|     M|  Silver|
|       15|   Alexandra Tabares|    MG|     F|  

In [19]:
# Vendo o tipo de arquivo é a variável clientes
type (clientes)

pyspark.sql.dataframe.DataFrame

In [20]:
# Visualizando via select as colunas "Clientes", "Estado" e "Status"
parte1 = clientes.select("Cliente","Estado","Status")
parte1.show()

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [22]:
# Filtrando por estado os clientes platinum e gold
parte2 = clientes.select("Estado","Status").where(clientes.Status != "Silver")
parte2.show()

+------+--------+
|Estado|  Status|
+------+--------+
|    RO|Platinum|
|    RN|    Gold|
|    RS|    Gold|
|    SP|    Gold|
|    AM|    Gold|
|    TO|    Gold|
|    PR|    Gold|
|    AM|    Gold|
|    CE|Platinum|
|    PE|    Gold|
|    RR|Platinum|
|    BA|    Gold|
|    GO|Platinum|
+------+--------+



In [24]:
df_left = vendas

In [25]:
df_right = clientes

In [29]:
# Fiz um join para juntar as duas tabelas pelo o ClienteID
parte3_1 = df_left.join(df_right, on=['ClienteID'], how='Inner').orderBy('ClienteID')
parte3_1.show()

+---------+--------+----------+----------+--------+--------------------+------+------+------+
|ClienteID|VendasID|VendedorID|      Data|   Total|             Cliente|Estado|Genero|Status|
+---------+--------+----------+----------+--------+--------------------+------+------+------+
|        1|     382|         7|13/12/2019| 9379.44|Adelina Buenaventura|    RJ|     M|Silver|
|        2|     254|         3| 22/8/2020| 5782.68|        Adelino Gago|    RJ|     M|Silver|
|        3|     336|         5| 7/11/2020| 10658.9|     Adolfo Patrício|    PE|     M|Silver|
|        5|     292|         7| 29/9/2020|13543.56|       Adélio Lisboa|    SE|     M|Silver|
|        5|     374|         1| 5/12/2019|17096.16|       Adélio Lisboa|    SE|     M|Silver|
|        5|     355|         5|23/11/2019|27016.24|       Adélio Lisboa|    SE|     M|Silver|
|        8|     378|         8|10/12/2018|   155.0|   Alarico Quinterno|    AC|     M|Silver|
|       14|     316|        10|25/10/2019|  253.58|        A

In [35]:
# Após a junção, separei por status a somatória de vendas
parte3_2 = parte3_1.groupBy("Status").agg(sum("VendasID")).orderBy(f.col("Sum(VendasID)").asc())
parte3_2.show()

+--------+-------------+
|  Status|sum(VendasID)|
+--------+-------------+
|Platinum|          719|
|    Gold|          985|
|  Silver|        78496|
+--------+-------------+

